# About this notebook

This notebook explores fire severity as classification problem using integrated dataset with 40 PCA and provides results based on two threshold recomneded to seperate labels 

## Loading the fire predictions

In [291]:
%matplotlib inline
import seaborn as sns
import pandas as pd
import numpy as np
sns.set_style('darkgrid')
import warnings
warnings.filterwarnings('ignore')

In [292]:
%%time
import pandas as pd
import pyarrow.parquet as pq
import s3fs
import boto3
import io

Wall time: 1.99 ms


In [293]:
# Accessing S3 bucket with fireonly integrateddata wiht pca40
s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3')
fireonly_url = 'wildfire_capstone/integratedData/integratedData.pca40_and_firesonly.parquet.gz/'
s3_keys = [item.key
           for item in s3_resource.Bucket('dse-cohort5-group5').objects.filter(Prefix=fireonly_url)
           if item.key.endswith('.parquet')]
display(s3_keys[:3])

['wildfire_capstone/integratedData/integratedData.pca40_and_firesonly.parquet.gz/part-00000-f1d8e022-8b57-43dc-a0dd-592bc9adf2ad-c000.gz.parquet',
 'wildfire_capstone/integratedData/integratedData.pca40_and_firesonly.parquet.gz/part-00001-f1d8e022-8b57-43dc-a0dd-592bc9adf2ad-c000.gz.parquet',
 'wildfire_capstone/integratedData/integratedData.pca40_and_firesonly.parquet.gz/part-00002-f1d8e022-8b57-43dc-a0dd-592bc9adf2ad-c000.gz.parquet']

In [294]:
len(s3_keys)

200

In [295]:
##anon=False
s3 = s3fs.S3FileSystem()   
s3_url = 'dse-cohort5-group5/{}'
dfs = [pq.ParquetDataset(s3_url.format(key), filesystem=s3).read_pandas().to_pandas() for key in s3_keys]


In [296]:
num_rows = sum(len(df) for df in dfs)
num_rows

2167

In [297]:
# Concat all dataframes into combined dataframe
dff = pd.concat(dfs)
dff.head()

,date,latitude,longitude,fire_occurred,acres_burned,year,month,day,pcaFeaturesArr[0],pcaFeaturesArr[1],...,pcaFeaturesArr[30],pcaFeaturesArr[31],pcaFeaturesArr[32],pcaFeaturesArr[33],pcaFeaturesArr[34],pcaFeaturesArr[35],pcaFeaturesArr[36],pcaFeaturesArr[37],pcaFeaturesArr[38],pcaFeaturesArr[39]
0,1067040000000000000,32.775000,-116.933333,1,270686.000000,2003,10,25,-1.279795,12.545055,...,-0.546615,0.039315,-1.049986,0.148678,0.140634,-0.628120,-0.615347,0.606489,-2.682956,-0.000420
1,1067040000000000000,32.941667,-116.766667,1,270686.000000,2003,10,25,8.553702,6.594267,...,-1.024007,0.032058,-0.831248,0.387239,0.570735,0.139129,0.109658,-0.161319,-1.332964,0.024806
2,1192924800000000000,32.691667,-116.891667,1,90728.328125,2007,10,21,1.144086,2.739561,...,-0.574740,0.223633,-1.104973,-0.054726,0.818893,0.589548,-0.112022,0.331131,0.078054,-0.036084
3,1193011200000000000,33.191667,-117.016667,1,164038.218000,2007,10,22,1.415704,1.031003,...,-0.125726,-1.066130,-0.631703,1.564491,0.816761,-0.657401,0.711758,-0.141134,0.443211,-0.159693
4,1193097600000000000,33.066667,-116.850000,1,108911.685000,2007,10,23,3.869830,-3.855695,...,-0.571720,-0.789562,-0.618795,0.994336,1.086224,-1.182717,-0.218400,0.231799,-0.308896,0.188512


In [298]:
dff.head()

,date,latitude,longitude,fire_occurred,acres_burned,year,month,day,pcaFeaturesArr[0],pcaFeaturesArr[1],...,pcaFeaturesArr[30],pcaFeaturesArr[31],pcaFeaturesArr[32],pcaFeaturesArr[33],pcaFeaturesArr[34],pcaFeaturesArr[35],pcaFeaturesArr[36],pcaFeaturesArr[37],pcaFeaturesArr[38],pcaFeaturesArr[39]
0,1067040000000000000,32.775000,-116.933333,1,270686.000000,2003,10,25,-1.279795,12.545055,...,-0.546615,0.039315,-1.049986,0.148678,0.140634,-0.628120,-0.615347,0.606489,-2.682956,-0.000420
1,1067040000000000000,32.941667,-116.766667,1,270686.000000,2003,10,25,8.553702,6.594267,...,-1.024007,0.032058,-0.831248,0.387239,0.570735,0.139129,0.109658,-0.161319,-1.332964,0.024806
2,1192924800000000000,32.691667,-116.891667,1,90728.328125,2007,10,21,1.144086,2.739561,...,-0.574740,0.223633,-1.104973,-0.054726,0.818893,0.589548,-0.112022,0.331131,0.078054,-0.036084
3,1193011200000000000,33.191667,-117.016667,1,164038.218000,2007,10,22,1.415704,1.031003,...,-0.125726,-1.066130,-0.631703,1.564491,0.816761,-0.657401,0.711758,-0.141134,0.443211,-0.159693
4,1193097600000000000,33.066667,-116.850000,1,108911.685000,2007,10,23,3.869830,-3.855695,...,-0.571720,-0.789562,-0.618795,0.994336,1.086224,-1.182717,-0.218400,0.231799,-0.308896,0.188512


In [305]:
dff.head()

,latitude,longitude,acres_burned,year,month,day,pcaFeaturesArr[0],pcaFeaturesArr[1],pcaFeaturesArr[2],pcaFeaturesArr[3],...,pcaFeaturesArr[30],pcaFeaturesArr[31],pcaFeaturesArr[32],pcaFeaturesArr[33],pcaFeaturesArr[34],pcaFeaturesArr[35],pcaFeaturesArr[36],pcaFeaturesArr[37],pcaFeaturesArr[38],pcaFeaturesArr[39]
0,32.775000,-116.933333,270686.000000,2003,10,25,-1.279795,12.545055,-6.556399,-4.858738,...,-0.546615,0.039315,-1.049986,0.148678,0.140634,-0.628120,-0.615347,0.606489,-2.682956,-0.000420
1,32.941667,-116.766667,270686.000000,2003,10,25,8.553702,6.594267,-5.784875,-2.088870,...,-1.024007,0.032058,-0.831248,0.387239,0.570735,0.139129,0.109658,-0.161319,-1.332964,0.024806
2,32.691667,-116.891667,90728.328125,2007,10,21,1.144086,2.739561,-5.606373,-1.414249,...,-0.574740,0.223633,-1.104973,-0.054726,0.818893,0.589548,-0.112022,0.331131,0.078054,-0.036084
3,33.191667,-117.016667,164038.218000,2007,10,22,1.415704,1.031003,-4.388951,0.429611,...,-0.125726,-1.066130,-0.631703,1.564491,0.816761,-0.657401,0.711758,-0.141134,0.443211,-0.159693
4,33.066667,-116.850000,108911.685000,2007,10,23,3.869830,-3.855695,-3.338547,-2.749853,...,-0.571720,-0.789562,-0.618795,0.994336,1.086224,-1.182717,-0.218400,0.231799,-0.308896,0.188512


In [299]:
# No need for fire occurred as this data is for fire only 
dff=dff.drop(['date', 'fire_occurred'], axis=1)
dff.shape

(2167, 46)

In [307]:
dff.columns

Index(['latitude', 'longitude', 'acres_burned', 'year', 'month', 'day',
       'pcaFeaturesArr[0]', 'pcaFeaturesArr[1]', 'pcaFeaturesArr[2]',
       'pcaFeaturesArr[3]', 'pcaFeaturesArr[4]', 'pcaFeaturesArr[5]',
       'pcaFeaturesArr[6]', 'pcaFeaturesArr[7]', 'pcaFeaturesArr[8]',
       'pcaFeaturesArr[9]', 'pcaFeaturesArr[10]', 'pcaFeaturesArr[11]',
       'pcaFeaturesArr[12]', 'pcaFeaturesArr[13]', 'pcaFeaturesArr[14]',
       'pcaFeaturesArr[15]', 'pcaFeaturesArr[16]', 'pcaFeaturesArr[17]',
       'pcaFeaturesArr[18]', 'pcaFeaturesArr[19]', 'pcaFeaturesArr[20]',
       'pcaFeaturesArr[21]', 'pcaFeaturesArr[22]', 'pcaFeaturesArr[23]',
       'pcaFeaturesArr[24]', 'pcaFeaturesArr[25]', 'pcaFeaturesArr[26]',
       'pcaFeaturesArr[27]', 'pcaFeaturesArr[28]', 'pcaFeaturesArr[29]',
       'pcaFeaturesArr[30]', 'pcaFeaturesArr[31]', 'pcaFeaturesArr[32]',
       'pcaFeaturesArr[33]', 'pcaFeaturesArr[34]', 'pcaFeaturesArr[35]',
       'pcaFeaturesArr[36]', 'pcaFeaturesArr[37]', 'pcaFeature

In [314]:
dfr=dff.drop(['acres_burned'], axis=1)
features=dfr.columns
features

Index(['latitude', 'longitude', 'year', 'month', 'day', 'pcaFeaturesArr[0]',
       'pcaFeaturesArr[1]', 'pcaFeaturesArr[2]', 'pcaFeaturesArr[3]',
       'pcaFeaturesArr[4]', 'pcaFeaturesArr[5]', 'pcaFeaturesArr[6]',
       'pcaFeaturesArr[7]', 'pcaFeaturesArr[8]', 'pcaFeaturesArr[9]',
       'pcaFeaturesArr[10]', 'pcaFeaturesArr[11]', 'pcaFeaturesArr[12]',
       'pcaFeaturesArr[13]', 'pcaFeaturesArr[14]', 'pcaFeaturesArr[15]',
       'pcaFeaturesArr[16]', 'pcaFeaturesArr[17]', 'pcaFeaturesArr[18]',
       'pcaFeaturesArr[19]', 'pcaFeaturesArr[20]', 'pcaFeaturesArr[21]',
       'pcaFeaturesArr[22]', 'pcaFeaturesArr[23]', 'pcaFeaturesArr[24]',
       'pcaFeaturesArr[25]', 'pcaFeaturesArr[26]', 'pcaFeaturesArr[27]',
       'pcaFeaturesArr[28]', 'pcaFeaturesArr[29]', 'pcaFeaturesArr[30]',
       'pcaFeaturesArr[31]', 'pcaFeaturesArr[32]', 'pcaFeaturesArr[33]',
       'pcaFeaturesArr[34]', 'pcaFeaturesArr[35]', 'pcaFeaturesArr[36]',
       'pcaFeaturesArr[37]', 'pcaFeaturesArr[38]', 'pcaF

In [315]:
yy=dff['year'].unique()
yy

array(['2003', '2007', '2011', '2012', '2013', '2005', '2009', '2015',
       '2016', '2002', '2010', '2014', '2017', '2006', '2004', '2008',
       '2001', '2018'], dtype=object)

In [317]:
dff.head()

,latitude,longitude,acres_burned,year,month,day,pcaFeaturesArr[0],pcaFeaturesArr[1],pcaFeaturesArr[2],pcaFeaturesArr[3],...,pcaFeaturesArr[30],pcaFeaturesArr[31],pcaFeaturesArr[32],pcaFeaturesArr[33],pcaFeaturesArr[34],pcaFeaturesArr[35],pcaFeaturesArr[36],pcaFeaturesArr[37],pcaFeaturesArr[38],pcaFeaturesArr[39]
0,32.775000,-116.933333,270686.000000,2003,10,25,-1.279795,12.545055,-6.556399,-4.858738,...,-0.546615,0.039315,-1.049986,0.148678,0.140634,-0.628120,-0.615347,0.606489,-2.682956,-0.000420
1,32.941667,-116.766667,270686.000000,2003,10,25,8.553702,6.594267,-5.784875,-2.088870,...,-1.024007,0.032058,-0.831248,0.387239,0.570735,0.139129,0.109658,-0.161319,-1.332964,0.024806
2,32.691667,-116.891667,90728.328125,2007,10,21,1.144086,2.739561,-5.606373,-1.414249,...,-0.574740,0.223633,-1.104973,-0.054726,0.818893,0.589548,-0.112022,0.331131,0.078054,-0.036084
3,33.191667,-117.016667,164038.218000,2007,10,22,1.415704,1.031003,-4.388951,0.429611,...,-0.125726,-1.066130,-0.631703,1.564491,0.816761,-0.657401,0.711758,-0.141134,0.443211,-0.159693
4,33.066667,-116.850000,108911.685000,2007,10,23,3.869830,-3.855695,-3.338547,-2.749853,...,-0.571720,-0.789562,-0.618795,0.994336,1.086224,-1.182717,-0.218400,0.231799,-0.308896,0.188512


In [323]:
# Calculate the accuracy of classes for each year for last 20 years. some years dont have any fires
# Create neasted dictionary to cover each year with accuracy , confusion matrix and classificaiton report
# Results include output which has actual and predicated classes plus all features for test dataset 
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import PredefinedSplit
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
dfmt=dff
output=[]
accuracy=[]
conf_matrx=[]
class_report=[]
grid_parametrs=[]
d={}

for i,year in enumerate(yyu):    
    dfmtt1=dfmt[dfmt['year']==year]
    print('Number of total fires of {} = '.format(year), dfmtt1.shape[0])
    dfmttest=pd.concat([dfmtt1])
    #Shuffle dataset with random state so it can be reproduced 
    dfmttest = dfmttest.sample(frac=1, axis=0, random_state=10).reset_index(drop=True)
    xt=dfmttest[features]
    yt=dfmttest['acres_burned']
    print('The number of fires of {} = {}'.format(year,len(yt)))
    dfmtt=dfmt[(dfmt['year'] != year)]
    dfmtt=dfmtt.sample(frac=1, axis=0, random_state=10).reset_index(drop=True)
    print('Number of total fires of rest of years = ', dfmtt.shape[0])
    x=dfmtt[features]
    y=dfmtt['acres_burned']
    print('The number of fires of {} = {}'.format(year,len(y)))
    # use 2004 as validation dataset to tune the parameters 
    dfmttain=dfmt[dfmt['year'] == '2004']
    ytrain=dfmttain['acres_burned'].values
    # Focus on randomforesetclassifier as it always outperform  Logistic regression, Xgboost and SVM 
    clf=RandomForestRegressor()
    ps = PredefinedSplit(ytrain)
    parameters = {'n_estimators': [15,20,25,50,100,150], 'max_depth' : [5,10,15,20,25,30],'min_samples_split':[1,2,3,4,6], 
                  'min_samples_leaf':[1,2,3,5,7],'random_state' : [5]}
    CV_rfc = GridSearchCV(clf , parameters,scoring= 'neg_mean_absolute_error', cv=ps)
    CV_rfc.fit(x, y)
    y_pred=CV_rfc.predict(xt)
    ypd=pd.DataFrame(y_pred, columns=['P_Acres_burned'])
    ytd=pd.Series.to_frame(yt, name='A_Acres_burned')
    outputdf=pd.concat([xt,ypd,ytd],axis=1)
    output.append(outputdf)
    vaild_acc= metrics.mean_absolute_error(yt, y_pred)
    print('The validation accuracy of year {} is= {} '.format(year,vaild_acc))
    accuracy.append(vaild_acc)
    grid_parametrs.append(CV_rfc.best_params_)
    d[year]={}
    d[year]['output']=output[i]
    d[year]['accuracy']=accuracy[i]
    d[year]['grid_parametrs']=grid_parametrs[i]

    

Number of total fires of 2001 =  17
The number of fires of 2001 = 17
Number of total fires of rest of years =  2150
The number of fires of 2001 = 2150
The validation accuracy of year 2001 is= 13210.804469623068 
Number of total fires of 2002 =  75
The number of fires of 2002 = 75
Number of total fires of rest of years =  2092
The number of fires of 2002 = 2092
The validation accuracy of year 2002 is= 33478.54084344297 
Number of total fires of 2003 =  174
The number of fires of 2003 = 174
Number of total fires of rest of years =  1993
The number of fires of 2003 = 1993
The validation accuracy of year 2003 is= 111217.69621953998 
Number of total fires of 2005 =  57
The number of fires of 2005 = 57
Number of total fires of rest of years =  2110
The number of fires of 2005 = 2110
The validation accuracy of year 2005 is= 5959.315164331613 
Number of total fires of 2006 =  73
The number of fires of 2006 = 73
Number of total fires of rest of years =  2094
The number of fires of 2006 = 2094
T

In [324]:
d[0]

KeyError: 0

In [325]:
for y in yyu:
    print('The accuracy of year {} is {}'.format(y,d[y]['accuracy']))


The accuracy of year 2001 is 13210.804469623068
The accuracy of year 2002 is 33478.54084344297
The accuracy of year 2003 is 111217.69621953998
The accuracy of year 2005 is 5959.315164331613
The accuracy of year 2006 is 11861.7984119149
The accuracy of year 2007 is 86244.50535734821
The accuracy of year 2008 is 6585.5896578594775
The accuracy of year 2009 is 6037.584866979594
The accuracy of year 2010 is 3352.135996572254
The accuracy of year 2011 is 27699.089263790742
The accuracy of year 2012 is 2888.310447650647
The accuracy of year 2013 is 3432.902098666766
The accuracy of year 2014 is 4407.658843984959
The accuracy of year 2015 is 1134.8154095855161
The accuracy of year 2016 is 3216.1587538964636
The accuracy of year 2017 is 10878.941295676914
The accuracy of year 2018 is 2401.0111914296917


In [326]:
dfst=[d[y]['output'] for y in yyu ]
dfst=pd.concat(dfst)
dfst.shape

(2114, 47)

In [327]:
dfst.to_parquet('fire_severity_randomforest_regression.parquet')

In [288]:
dfst.write.parquet(
    's3a://dse-cohort5-group5/wildfire_capstone/fire_severity_classification_prediction/fire_severity_Regression_result.parquet.gz',
    partitionBy=["year"],
    mode="overwrite",
    compression='gzip')

AttributeError: 'DataFrame' object has no attribute 'write'

In [328]:
dfst.to_csv('output_Regression_RandomforestRegressor.csv')